###**Descrição da ponderada:**

Objetivo: Otimizar um modelo de rede neural pré-treinado para detecção de fraudes em cartões de crédito. Aplicar técnicas avançadas de ajuste fino de hiperparâmetros, como grid search e random search, com o objetivo de aprimorar as métricas de desempenho do modelo, incluindo precisão, recall, F1-score e AUC-ROC. A atividade também exige uma comparação entre o modelo otimizado e o modelo original, permitindo avaliar o impacto das modificações nos hiperparâmetros sobre o desempenho geral.

In [1]:
%pip install gdown
import gdown

In [2]:
arquivo_destino_colab = "dataset.csv"
doc_id = "1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl"
URL = f"https://drive.google.com/uc?id={doc_id}"
gdown.download(URL, arquivo_destino_colab, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl
From (redirected): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl&confirm=t&uuid=c9f17738-9fbf-46c8-be5d-89da0b06d93d
To: /content/dataset.csv
100%|██████████| 151M/151M [00:01<00:00, 134MB/s]


'dataset.csv'

In [18]:
import os, time, math, random, numpy as np, pandas as pd
from itertools import product
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

SEED = 42
np.random.seed(SEED); random.seed(SEED); tf.random.set_seed(SEED)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"


In [19]:
PATH = "dataset.csv"
df = pd.read_csv(PATH)
cands = ["Class","class","is_fraud","fraud","Fraud","target","label","y"]
target_col = next((c for c in cands if c in df.columns), df.columns[-1])
y = df[target_col].astype(int) if df[target_col].dtype != 'int' else df[target_col]
X = df.drop(columns=[target_col])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

num_sel = selector(dtype_include=np.number)
cat_sel = selector(dtype_exclude=np.number)

num_pipe = Pipeline([("scaler", StandardScaler())])
cat_pipe = Pipeline([("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))])
preprocessor = ColumnTransformer(
    [("num", num_pipe, num_sel(X_train)), ("cat", cat_pipe, cat_sel(X_train))],
    remainder="drop"
)
preprocessor.fit(X_train)
X_train_t = preprocessor.transform(X_train)
X_test_t  = preprocessor.transform(X_test)
n_features = X_train_t.shape[1]

classes = np.unique(y_train)
cw = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weight_dict = {int(k): float(v) for k, v in zip(classes, cw)}

n_features


30

In [20]:
def build_mlp(input_dim, hidden1=64, hidden2=32, dropout=0.3, learning_rate=1e-3, optimizer="adam"):
    inp = keras.Input(shape=(input_dim,))
    x = layers.Dense(hidden1, activation="relu")(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(hidden2, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout)(x)
    out = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inp, out)
    opt = {
        "adam": keras.optimizers.Adam(learning_rate=learning_rate),
        "adamax": keras.optimizers.Adamax(learning_rate=learning_rate),
        "sgd": keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True),
    }.get(optimizer.lower(), keras.optimizers.Adam(learning_rate=learning_rate))
    model.compile(optimizer=opt, loss="binary_crossentropy", metrics=[keras.metrics.AUC(name="auc")])
    return model


In [21]:
baseline = build_mlp(n_features, hidden1=64, hidden2=32, dropout=0.3, learning_rate=1e-3, optimizer="adam")
callbacks = [
    keras.callbacks.EarlyStopping(monitor="loss", patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=3, verbose=0),
]
baseline.fit(X_train_t, y_train, epochs=12, batch_size=2048, class_weight=class_weight_dict, verbose=0, callbacks=callbacks)

PRETRAINED_PATH = "/content/pretrained.weights.h5"
baseline.save_weights(PRETRAINED_PATH)

y_prob_base = baseline.predict(X_test_t, verbose=0).ravel()
y_pred_base = (y_prob_base >= 0.5).astype(int)
base_metrics = {
    "precision": precision_score(y_test, y_pred_base, zero_division=0),
    "recall": recall_score(y_test, y_pred_base, zero_division=0),
    "f1": f1_score(y_test, y_pred_base, zero_division=0),
    "auc_roc": roc_auc_score(y_test, y_prob_base),
}
base_metrics


{'precision': 0.08644400785854617,
 'recall': 0.8979591836734694,
 'f1': 0.15770609318996415,
 'auc_roc': np.float64(0.9739049777198443)}

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

def fold_preprocess(X, y, train_idx, val_idx):
    X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]
    pre = ColumnTransformer(
        [("num", StandardScaler(), num_sel(X_tr)), ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_sel(X_tr))],
        remainder="drop"
    )
    pre.fit(X_tr)
    Xt, Xv = pre.transform(X_tr), pre.transform(X_va)
    return Xt, y_tr.values, Xv, y_va.values

def class_weight_for(y):
    cls = np.unique(y)
    w = compute_class_weight(class_weight="balanced", classes=cls, y=y)
    return {int(k): float(v) for k, v in zip(cls, w)}

def train_eval(params, Xt, yt, Xv, yv, class_weight):
    tf.keras.backend.clear_session()
    m = build_mlp(
        input_dim=Xt.shape[1],
        hidden1=params["hidden1"], hidden2=params["hidden2"],
        dropout=params["dropout"], learning_rate=params["learning_rate"],
        optimizer=params["optimizer"]
    )
    if os.path.exists(PRETRAINED_PATH):
        try:
            m.load_weights(PRETRAINED_PATH)
        except Exception:
            pass
    cbs = [
        keras.callbacks.EarlyStopping(monitor="loss", patience=4, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=2, verbose=0),
    ]
    m.fit(Xt, yt, epochs=params["epochs"], batch_size=params["batch_size"], class_weight=class_weight, verbose=0, callbacks=cbs)
    yp = m.predict(Xv, verbose=0).ravel()
    return roc_auc_score(yv, yp)


In [23]:
param_grid = {
    "hidden1": [64, 128],
    "hidden2": [32],
    "dropout": [0.25, 0.4],
    "learning_rate": [1e-3, 5e-4],
    "optimizer": ["adam", "adamax"],
    "epochs": [12],
    "batch_size": [1024],
}

keys = list(param_grid.keys())
combos = [dict(zip(keys, v)) for v in product(*param_grid.values())]
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)

best_grid = {"score": -1, "params": None}
t0 = time.time()
for combo in combos:
    scores = []
    for tr, va in skf.split(X_train, y_train):
        Xt, yt, Xv, yv = fold_preprocess(X_train, y_train, tr, va)
        cw = class_weight_for(yt)
        s = train_eval(combo, Xt, yt, Xv, yv, cw)
        scores.append(s)
    mean_s = float(np.mean(scores))
    if mean_s > best_grid["score"]:
        best_grid["score"] = mean_s
        best_grid["params"] = combo
t_grid = time.time() - t0
best_grid, round(t_grid,1)


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adamax', because it has 2 variables whereas the saved 

({'score': 0.9970727784281226,
  'params': {'hidden1': 64,
   'hidden2': 32,
   'dropout': 0.25,
   'learning_rate': 0.0005,
   'optimizer': 'adamax',
   'epochs': 12,
   'batch_size': 1024}},
 1016.4)

In [24]:
bp = best_grid["params"]
tf.keras.backend.clear_session()
pre = ColumnTransformer(
    [("num", StandardScaler(), num_sel(X_train)), ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_sel(X_train))],
    remainder="drop"
)
pre.fit(X_train)
Xtr_full = pre.transform(X_train)
Xte_full = pre.transform(X_test)
cw_full = class_weight_for(y_train.values)

m_grid = build_mlp(Xtr_full.shape[1], bp["hidden1"], bp["hidden2"], bp["dropout"], bp["learning_rate"], bp["optimizer"])
if os.path.exists(PRETRAINED_PATH):
    try:
        m_grid.load_weights(PRETRAINED_PATH)
    except Exception:
        pass
cbs = [
    keras.callbacks.EarlyStopping(monitor="loss", patience=4, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=2, verbose=0),
]
m_grid.fit(Xtr_full, y_train.values, epochs=bp["epochs"], batch_size=bp["batch_size"], class_weight=cw_full, verbose=0, callbacks=cbs)
y_prob_grid = m_grid.predict(Xte_full, verbose=0).ravel()
y_pred_grid = (y_prob_grid >= 0.5).astype(int)
grid_metrics = {
    "precision": precision_score(y_test, y_pred_grid, zero_division=0),
    "recall": recall_score(y_test, y_pred_grid, zero_division=0),
    "f1": f1_score(y_test, y_pred_grid, zero_division=0),
    "auc_roc": roc_auc_score(y_test, y_prob_grid),
}
grid_metrics


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adamax', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


{'precision': 0.10220673635307782,
 'recall': 0.8979591836734694,
 'f1': 0.1835245046923879,
 'auc_roc': np.float64(0.9745064845015102)}

In [25]:
def sample_params(space):
    return {
        "hidden1": int(np.random.randint(space["hidden1"][0], space["hidden1"][1]+1)),
        "hidden2": int(np.random.randint(space["hidden2"][0], space["hidden2"][1]+1)),
        "dropout": float(np.random.uniform(space["dropout"][0], space["dropout"][1])),
        "learning_rate": float(np.exp(np.random.uniform(np.log(space["lr"][0]), np.log(space["lr"][1])))),
        "optimizer": random.choice(space["optimizer"]),
        "epochs": int(np.random.randint(space["epochs"][0], space["epochs"][1]+1)),
        "batch_size": int(random.choice(space["batch_size"])),
    }

space = {
    "hidden1": (64, 192),
    "hidden2": (16, 96),
    "dropout": (0.2, 0.5),
    "lr": (3e-4, 1.5e-3),
    "optimizer": ["adam","adamax","sgd"],
    "epochs": (10, 18),
    "batch_size": [1024, 2048],
}

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
best_rand = {"score": -1, "params": None}
t0 = time.time()
for _ in range(10):
    combo = sample_params(space)
    scores = []
    for tr, va in skf.split(X_train, y_train):
        Xt, yt, Xv, yv = fold_preprocess(X_train, y_train, tr, va)
        cw = class_weight_for(yt)
        s = train_eval(combo, Xt, yt, Xv, yv, cw)
        scores.append(s)
    mean_s = float(np.mean(scores))
    if mean_s > best_rand["score"]:
        best_rand["score"] = mean_s
        best_rand["params"] = combo
t_rand = time.time() - t0
best_rand, round(t_rand,1)


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimi

({'score': 0.9789044497669056,
  'params': {'hidden1': 116,
   'hidden2': 17,
   'dropout': 0.41659963168004743,
   'learning_rate': 0.0013587559091193951,
   'optimizer': 'sgd',
   'epochs': 11,
   'batch_size': 2048}},
 904.9)

In [26]:
rp = best_rand["params"]
tf.keras.backend.clear_session()
Xtr_full = pre.transform(X_train)
Xte_full = pre.transform(X_test)
cw_full = class_weight_for(y_train.values)

m_rand = build_mlp(Xtr_full.shape[1], rp["hidden1"], rp["hidden2"], rp["dropout"], rp["learning_rate"], rp["optimizer"])
if os.path.exists(PRETRAINED_PATH):
    try:
        m_rand.load_weights(PRETRAINED_PATH)
    except Exception:
        pass
cbs = [
    keras.callbacks.EarlyStopping(monitor="loss", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=2, verbose=0),
]
m_rand.fit(Xtr_full, y_train.values, epochs=rp["epochs"], batch_size=rp["batch_size"], class_weight=cw_full, verbose=0, callbacks=cbs)
y_prob_rand = m_rand.predict(Xte_full, verbose=0).ravel()
y_pred_rand = (y_prob_rand >= 0.5).astype(int)
rand_metrics = {
    "precision": precision_score(y_test, y_pred_rand, zero_division=0),
    "recall": recall_score(y_test, y_pred_rand, zero_division=0),
    "f1": f1_score(y_test, y_pred_rand, zero_division=0),
    "auc_roc": roc_auc_score(y_test, y_prob_rand),
}
rand_metrics


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


{'precision': 0.06395348837209303,
 'recall': 0.8979591836734694,
 'f1': 0.11940298507462686,
 'auc_roc': np.float64(0.9794995111860164)}

In [27]:
comparacao = pd.DataFrame([
    {"modelo":"baseline", **base_metrics},
    {"modelo":"grid_best", **grid_metrics},
    {"modelo":"random_best", **rand_metrics},
]).set_index("modelo").sort_values("auc_roc", ascending=False)
comparacao


,precision,recall,f1,auc_roc
modelo,,,,
random_best,0.063953,0.897959,0.119403,0.979500
grid_best,0.102207,0.897959,0.183525,0.974506
baseline,0.086444,0.897959,0.157706,0.973905
